In [ ]:
import sys
sys.path.append('../')
from datasets import load_dataset
from transformers import AutoTokenizer
from src.frameworks import TruncatedMC

In [ ]:
dataset = load_dataset("yelp_review_full")

In [ ]:
dataset["train"][2]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [ ]:
type(small_train_dataset)

In [ ]:
type(small_train_dataset['text'])

The following cell output should contain a warning like: "Some weights of the model checkpoint at bert-base-cased were not used when initializing...".

This is OK as we are not using the pooler layer in order to compute the loss.

In [ ]:
dshap = TruncatedMC(train_dataset=small_train_dataset, X_train=small_train_dataset['text'], X_test=small_eval_dataset['text'], test_dataset=small_eval_dataset, model_family='bert-base-cased', num_labels=5)

Print the initial accuracy

In [ ]:
dshap.random_score

In [ ]:
dshap.run(save_every=100, err=0.1, do_loo=True, do_tmc=False, do_gshap=False)